In [ ]:
print("Running setup...")
!pip install langgraph --quiet

Running setup...


### Part 1: The Problem - Manual State Updates

Let's look at the standard way we've been updating state. Each node is responsible for returning a dictionary with the specific keys it wants to change. This works, but can get messy.

In [ ]:
from typing import TypedDict, List

class ManualState(TypedDict):
    tasks: List[str]
    completed_tasks: List[str]
    step: int

def add_task(state: ManualState) -> dict:
    print("NODE: Adding a task")
    # Manually update the 'tasks' list and 'step' counter
    new_tasks = state.get("tasks", []) + ["New Task"]
    return {"tasks": new_tasks, "step": state.get("step", 0) + 1}

def complete_task(state: ManualState) -> dict:
    print("NODE: Completing a task")
    tasks = state.get("tasks", [])
    if not tasks:
        return {}

    completed_task = tasks[0]
    remaining_tasks = tasks[1:]
    completed_list = state.get("completed_tasks", []) + [completed_task]

    # Manually update three different keys
    return {
        "tasks": remaining_tasks,
        "completed_tasks": completed_list,
        "step": state.get("step", 0) + 1
    }

initial_state = {"tasks": [], "completed_tasks": [], "step": 0}

# Simulate running the nodes
state_after_add = {**initial_state, **add_task(initial_state)}
print(f"State after add: {state_after_add}")

state_after_complete = {**state_after_add, **complete_task(state_after_add)}
print(f"State after complete: {state_after_complete}")

NODE: Adding a task
State after add: {'tasks': ['New Task'], 'completed_tasks': [], 'step': 1}
NODE: Completing a task
State after complete: {'tasks': [], 'completed_tasks': ['New Task'], 'step': 2}


### Part 2: Building a Reducer Function

Now, let's centralize all that logic. We'll define a single reducer function. This function will inspect an "action" and decide how to update the state. The nodes will now return **actions** instead of state updates.

In [ ]:
from typing import TypedDict, List, Union
import copy

# Define the possible actions our system can take
class AddTaskAction(TypedDict):
    type: str # Should be 'ADD_TASK'
    payload: str

class CompleteTaskAction(TypedDict):
    type: str # Should be 'COMPLETE_TASK'

Action = Union[AddTaskAction, CompleteTaskAction]

# Define the reducer function
def state_reducer(state: ManualState, action: Action) -> ManualState:
    """Takes the current state and an action, and returns the new state."""
    # Always work on a copy to ensure immutability
    new_state = copy.deepcopy(state)
    new_state["step"] += 1

    if action['type'] == 'ADD_TASK':
        new_state['tasks'].append(action['payload'])

    elif action['type'] == 'COMPLETE_TASK':
        if new_state['tasks']:
            task_to_complete = new_state['tasks'].pop(0)
            new_state['completed_tasks'].append(task_to_complete)

    return new_state

# Simulate the flow
current_state = {"tasks": [], "completed_tasks": [], "step": 0}
print(f"Initial State: {current_state}")

# Create an action to add a task
add_action = {"type": "ADD_TASK", "payload": "Write the report"}
current_state = state_reducer(current_state, add_action)
print(f"After ADD_TASK: {current_state}")

# Create an action to complete a task
complete_action = {"type": "COMPLETE_TASK"}
current_state = state_reducer(current_state, complete_action)
print(f"After COMPLETE_TASK: {current_state}")

Initial State: {'tasks': [], 'completed_tasks': [], 'step': 0}
After ADD_TASK: {'tasks': ['Write the report'], 'completed_tasks': [], 'step': 1}
After COMPLETE_TASK: {'tasks': [], 'completed_tasks': ['Write the report'], 'step': 2}


### Part 3: Using a Reducer in a LangGraph `Graph`

LangGraph's `Graph` class (as opposed to `StateGraph`) is designed to work perfectly with the reducer pattern. Instead of adding nodes that return dictionaries, you add a central reducer and your nodes can return **actions**.

The graph will automatically call your reducer with the output of each node.

In [ ]:
from langgraph.graph import StateGraph
from typing import TypedDict, List, Union
import copy

# -----------------------------
# Types and Reducer
# -----------------------------

class ManualState(TypedDict, total=False):
    tasks: List[str]
    completed_tasks: List[str]
    step: int
    action: dict  # optional key to hold current action

class AddTaskAction(TypedDict):
    type: str  # 'ADD_TASK'
    payload: str

class CompleteTaskAction(TypedDict):
    type: str  # 'COMPLETE_TASK'

Action = Union[AddTaskAction, CompleteTaskAction]

def state_reducer(state: ManualState, action: Action) -> ManualState:
    new_state = copy.deepcopy(state)
    new_state["step"] += 1

    if action["type"] == "ADD_TASK":
        new_state["tasks"].append(action["payload"])
    elif action["type"] == "COMPLETE_TASK":
        if new_state["tasks"]:
            completed = new_state["tasks"].pop(0)
            new_state["completed_tasks"].append(completed)

    return new_state

# -----------------------------
# Nodes
# -----------------------------

def plan_next_step(state: ManualState) -> ManualState:
    print("NODE: Planning next step...")
    if not state["tasks"]:
        action = {"type": "ADD_TASK", "payload": "First, define the project scope."}
    else:
        action = {"type": "COMPLETE_TASK"}

    return {**state, "action": action}

def apply_reducer(state: ManualState) -> ManualState:
    print(f"NODE: Reducing with action {state['action']}")
    new_state = state_reducer(state, state["action"])
    new_state.pop("action", None)  # Clean up
    return new_state

def should_continue(state: ManualState) -> str:
    if len(state["completed_tasks"]) >= 2:
        return "end"
    return "continue"

# -----------------------------
# Graph Assembly
# -----------------------------

builder = StateGraph(ManualState)

builder.add_node("planner", plan_next_step)
builder.add_node("reducer", apply_reducer)

builder.set_entry_point("planner")
builder.add_edge("planner", "reducer")

builder.add_conditional_edges(
    "reducer",
    should_continue,
    {
        "continue": "planner",
        "end": "__end__"
    }
)

app = builder.compile()

# -----------------------------
# Run the app
# -----------------------------

initial_state: ManualState = {
    "tasks": [],
    "completed_tasks": [],
    "step": 0
}

final_state = app.invoke(initial_state)

print("\n--- Final State from Graph ---")
print(final_state)


NODE: Planning next step...
NODE: Reducing with action {'type': 'ADD_TASK', 'payload': 'First, define the project scope.'}
NODE: Planning next step...
NODE: Reducing with action {'type': 'COMPLETE_TASK'}
NODE: Planning next step...
NODE: Reducing with action {'type': 'ADD_TASK', 'payload': 'First, define the project scope.'}
NODE: Planning next step...
NODE: Reducing with action {'type': 'COMPLETE_TASK'}

--- Final State from Graph ---
{'tasks': [], 'completed_tasks': ['First, define the project scope.', 'First, define the project scope.'], 'step': 4, 'action': {'type': 'COMPLETE_TASK'}}
